# Week1

- Recurrent Neural Networks
- GRU
- LSTM
- Deep RNN

## RNN

序列模型是输入一个序列输出一个序列的模型, 传统DNN不能解决输入长度与输出长度不统一的场景, 也不能解决在语言这类序列模型当中学习到位置信息并进行长程传递的问题, 即不能把一部分学到的东西快速的泛化到其他位置. RNN, 是为了解决序列模型的问题提出的模型

### 基本结构

<img src="src/pictures/c5_w1_1.jpg" width=50%>

RNN使用同一套纵向结构, 在时间维度上移动, 构成整体网络, 不同时间点共用同一套权重, 前面计算得到隐藏层值作为输入加入到后面时间点的计算中.

在每个时间步中, 一个神经元内的计算如下:

$$a_t = g_1(W_{aa}a_{t-1} + W_{ax}X + b) \leftarrow tanh \\
\hat{y_t} = g_2(W_{ya}a_t + b_y) \leftarrow bin/softmax $$

In [12]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']

    a_next = np.tanh(Wax.dot(xt) + Waa.dot(a_prev) + ba)
    yt_pred = softmax(Wya.dot(a_next) + by)

    cache = (a_next, a_prev, xt, parameters) # what I need

    return a_next, yt_pred, cache

### 前向传播

在前向传播中, 上一个时间步的隐藏层状态作为输入传入下一个时间步

In [13]:
import numpy as np

def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']
    
    n_x, m, T_x = x.shape
    n_a = a0.shape[0]
    n_y = by.shape[0]
    at = a0
    y_pred = np.zeros((n_y, m, T_x))
    a = np.zeros((n_a, m, T_x))
    caches = []

    for t in range(T_x):
        a[:, :, t], y_pred[:, :, t], cache = rnn_cell_forward(x[:, :, t], at, parameters)
        at = a[:, :, t]
        caches.append(cache)
    caches = [caches, x]

    return a, y_pred, caches

#### Cost函数

对于分类问题的RNN, 依然可以使用交叉熵作为Loss的计算方法, 则cost的计算为
$$L=\frac{1}{T}\sum_{t=1}^{T}l(y_t, \hat{y}_t) \\
\quad = -\frac{1}{T}\sum_{t=1}^{T} \sum_i y_t^{(i)}log \hat{y}_t^{(i)}$$

#### 反向传播BPTT

RNN的反向传播使用的很经典的BPTT方法(Back Propagation Through Time).

首先, 下图列出RNN结构的正向传播的数据流动方向, 反向传播时, 就是沿着正向的方向, 使用链式求导法则反向计算.

<img src="src/pictures/c5_w1_2.png" width=80%>

上图中, $o^{t} = W_{ya}a_t + b_y$, 忽略$b_y$则有$o^{t} = V a_t$,相似的, 为简单起见忽略$a_t$计算中的偏置, 则可以写成, $a_t = g_1(Wa_{t-1} + UX)$. 总结一下, 在上图中, $V = W_{ya},W=W_{aa}, U=W_{ax}$

L对于所有$\hat{y}_t$的求导均可写成
$$\frac{\partial{L}}{\partial{\hat{y}_t}} = \frac{\partial l(y_t, \hat{y}_t)}{T \partial \hat{y}_t}$$

因此, $W_{ya}$和$a_t$可以按照链式求导规则写成:

$$\frac{\partial{L}}{\partial{W_{ya}}}  = \sum_{t=1}^T \frac{\partial{L}}{\partial{\hat{y}_t}} \frac{\partial \hat{y}_t}{\partial W_{ya}} = \sum_{t=1}^T \frac{\partial L}{\partial \hat{y}_t} a_t^T \\
\frac{\partial{L}}{\partial{a_t}}  = \frac{\partial{L}}{\partial{\hat{y}_t}} \frac{\partial \hat{y}_t}{\partial a_t} = W_{ya}^T \frac{\partial L}{\partial \hat{y}_t}$$

$W_{aa}$和$W_{ax}$的梯度也可以写成:
$$\frac{\partial{L}}{\partial{W_{aa}}}  = \sum_{t=1}^T \frac{\partial{L}}{\partial a_t} \frac{\partial a_t}{\partial W_{aa}} \\
\frac{\partial{L}}{\partial{W_{ax}}}  = \sum_{t=1}^T \frac{\partial{L}}{\partial a_t} \frac{\partial a_t}{\partial W_{ax}} $$

如果在上图中,把$o_t$看做本层向上方层的输出, 在本层中, $W_{ya}$的梯度是容易求的, 而$W_{aa}$和$W_{ax}$的梯度均与$\frac{\partial{L}}{\partial a_t}$有关, 若已知$a_t$的梯度, 两个参数的梯度可以通过求$a_t$对它们的偏导得到.

$$\frac{\partial a_t}{\partial W_{aa}} = g_1^\prime a_{t-1}^T \\
\frac{\partial a_t}{\partial W_{ax}} = g_1^\prime x_t^T \\
\frac{\partial a_t}{\partial x_t} = W_{ax}^T g_1^{\prime} \\
\frac{\partial a_t}{\partial a_{t-1}} = W_{aa}^T g_1^\prime$$

从计算图中可以看出$\frac{\partial{L}}{\partial a_t}$来自两个方向, 上方网络和右侧网络, 因此
$$\frac{\partial{L}}{\partial a_t} = \frac{\partial{L}}{\partial{\hat{y}_t}} \frac{\partial \hat{y}_t}{\partial a_t} + \frac{\partial{L}}{\partial a_{t+1}} \frac{\partial a_{t+1}}{\partial a_{t}}$$

$W_{aa}$和$W_{ax}$的梯度最终可以写成:

$$\frac{\partial{L}}{\partial{W_{aa}}}  = \sum_{t=1}^T ( \frac{\partial{L}}{\partial{\hat{y}_t}} \frac{\partial \hat{y}_t}{\partial a_t} + \frac{\partial{L}}{\partial a_{t+1}} \frac{\partial a_{t+1}}{\partial a_{t}}) g_1^\prime a_{t-1}^T \\
\frac{\partial{L}}{\partial{W_{ax}}}  = \sum_{t=1}^T ( \frac{\partial{L}}{\partial{\hat{y}_t}} \frac{\partial \hat{y}_t}{\partial a_t} + \frac{\partial{L}}{\partial a_{t+1}} \frac{\partial a_{t+1}}{\partial a_{t}}) g_1^\prime x_t^T$$

In [14]:
def rnn_cell_backward(da_next, cache):
    """
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradients of input data, of shape (n_x, m)
                        da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dba -- Gradients of bias vector, of shape (n_a, 1)
    """
    a_next, a_prev, xt, parameters = cache
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']

    # Loss对Wax和Waa的求导 等于 Loss对at的求导 乘以 at对Zt的求导 乘以 Zt对W的求导
    dtanh = da_next * (1 - a_next**2) # (n_a, m) 

    dWax = np.dot(dtanh, xt.T) # Zt对Wax的求导 # (n_a, n_x)
    dWaa = np.dot(dtanh, a_prev.T) # (n_a, n_a)
    dba = np.sum(dtanh, axis=1, keepdims=True) # (n_a, 1)
    dx = np.dot(Wax.T, dtanh) # (n_x, m) 纵向具有多层时 需要
    da_prev = np.dot(Waa.T, dtanh) # (n_a, m) 横向的传播 这时将a_t-1 看做一个变量而非函数, 因为之后链式求导它还会继续乘下去然后加到最终的结果里, 总之, 针对dWaa的求导不用急着把a_t-1展开, 只要有对它的求导就够了 
    '''
    在这个函数中, 只处理a_t对W等的求导, 纵向上从上方传来的对a_t的求导和横向上从右边传过来的对a_t的求导, 相加后作为Cost对a_t的求导整体, 再乘以at对以上变量的求导
    '''
    gradients = {
        'dx':dx,
        'da_prev':da_prev,
        'dWax':dWax,
        'dWaa':dWaa,
        'dba':dba
    }
    return gradients

def rnn_backward(da, caches):
    """
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """
    caches, x = caches # caches -- list, 包含所有时间步的cache, x -- (n_x, m, T_x) 输入维度
    a1, a0, x1, parameters = caches[0]
    n_x, m, T_x = x.shape
    n_a = a1.shape[0]

    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da_prevt = np.zeros((n_a, m))

    for t in reversed(range(T_x)):
        # 在这里传入da_next的时候, 传入从右方和从上方传来的对Cost对a_t的求导之和, 求得的t时刻的梯度, 需要累加起来, 因为Cost是对所有t的累加
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients['dx'], gradients['da_prev'], gradients['dWax'], gradients['dWaa'], gradients['dba']
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
    da0 = da_prevt

    gradients = {
        'dx': dx, 
        'da0': da0,
        'dWax': dWax, 
        'dWaa': dWaa, 
        'dba': dba
    }
    return gradients

以上完成了一层的RNN的反向传播, 其中x的梯度可以作为下层RNN反向传播的输入, $\frac{\partial{L}}{\partial{\hat{y}}}$来自上层RNN的反向传播输出

In [15]:
def softmax(x):
    """
    Calculate the softmax value of x on 0 dim

    Arguments:
    x -- np.ndarray of (m, 1) dimension
    Returns:
    ret -- the softmax value, shape of (m, 1) 
    """
    e = np.exp(x)
    total = np.sum(e, axis=0)
    ret = e / total
    return ret

np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Wax = np.random.randn(5,3)
Waa = np.random.randn(5,5)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
a, y, caches = rnn_forward(x, a0, parameters)
da = np.random.randn(5, 10, 4)
gradients = rnn_backward(da, caches)

print("gradients[\"dx\"][1][2] =", gradients["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients["da0"].shape)
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)
print("gradients[\"dba\"][4] =", gradients["dba"][4])
print("gradients[\"dba\"].shape =", gradients["dba"].shape)

gradients["dx"][1][2] = [-2.07101689 -0.59255627  0.02466855  0.01483317]
gradients["dx"].shape = (3, 10, 4)
gradients["da0"][2][3] = -0.31494237512664996
gradients["da0"].shape = (5, 10)
gradients["dWax"][3][1] = 11.264104496527777
gradients["dWax"].shape = (5, 3)
gradients["dWaa"][1][2] = 2.303333126579893
gradients["dWaa"].shape = (5, 5)
gradients["dba"][4] = [-0.74747722]
gradients["dba"].shape = (5, 1)


### 梯度消失问题

由于BPTT中链式求导的大量连乘结构, 前方的梯度经过不断的连乘容易出现梯度消失和梯度爆炸的问题, 虽然RNN在不同的时间步是共享权重的, 后方的梯度就可以更新前方使用的权重, 但是仅有局部信息依然是不能够获得全局最优的. 针对梯度爆炸的问题, 可以使用梯度裁剪的方法解决, 但是梯度消失的问题难以处理, 为了解决梯度消失的问题, 研究者提出了更强大的RNN结构.

## GRU

<img src="src/pictures/c5_w1_3.png">

GRU相比RNN增加了上方的细胞状态, 在GRU中, 细胞状态与隐藏层状态是同一个, 在GRU中, 隐藏层状态不再有上一个时间步的隐藏层和本层的输入直接计算得到, 而是一直悬在上方,仅通过$z_t$门控制的门与输入信息交叉,进行更新, 这种结构使得隐藏层状态更加问题, 更容易记载长程信息不丢失.

$r_t$门反映了$h_{t-1}$和$\hat{h}_t$之间的相关性, $z_t$门决定了是否更新$h_t$

In [11]:
# implement GRU in pytorch

import torch
import torch.nn as nn
import torch.nn.functional as F

class GRUCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=False):
        '''
        Create a GRU cell.
        
        bias -- Not implement yet.
        '''
        super(GRUCell, self).__init__()
        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size))
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.input_to_hidden = nn.Linear(input_size, hidden_size * 2)
        self.hidden_to_hidden = nn.Linear(hidden_size, hidden_size * 2)
        
    def forward(self, xt, h_prevt):
        '''
        Do forward calculation.
        
        Arguments:
        xt -- input of t step, (batch, input_size)
        h_prevt -- hidden state of previous time step, (batch, hidden_size)
        
        Returns:
        ht -- hidden state of current time step, (batch, hidden_size)
        '''
        gates = self.input_to_hidden(xt) + self.hidden_to_hidden(h_prevt) # linear transform in zt, rt (batch, hidden_size*2)
        z_linear, r_linear = gates.chunk(2, 1) # seperate the results, (batch, hidden_size)
        zt = F.sigmoid(z_linear)
        rt = F.sigmoid(r_linear)
        h_hat = F.tanh(F.linear(xt, self.Wx) + F.linear(rt.mul(h_prevt), self.Wh)) # (batch, hidden_size)
        ht = (1 - zt).mul(h_prevt) + zt.mul(h_hat) # (batch, hidden_size)
        return ht
def test():        
    input_size = 5
    hidden_size = 10
    cell = GRUCell(input_size, hidden_size)

    xt = torch.rand(3, input_size)
    h_prev = torch.rand(3, hidden_size)

    ht = cell(xt, h_prev)
    print(ht)

test()

tensor([[ 0.3029,  0.2007,  0.9213,  0.0971,  0.4833,  0.5571,  0.7885,  0.5270,
         -0.1164,  0.4127],
        [ 0.3895, -0.4079,  0.8539,  0.8129,  0.7405,  0.5018,  0.8413,  0.5252,
          0.5424,  0.7923],
        [ 0.7851,  0.4036,  0.0778,  0.3468,  0.3874,  0.9650,  0.5882,  0.6734,
          0.8040,  0.3704]], grad_fn=<AddBackward0>)


## LSTM

LSTM的结构与GRU相似, 事实上GRU是LSTM的简化版, LSTM通过更新门, 遗忘门和输出门三个门控制细胞状态的更新和输出值, 由于和GRU相似的原因, 细胞状态能够更加容易的长时间的保持信息. LSTM的结构如下:

<img src="src/pictures/c5_w1_4.png">

LSTM的计算过程如下:

<img src="src/pictures/c5_w1_5.png">

In [10]:
# Implement LSTM in Pytroch

import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMCell, self).__init__()
        
        self.input2hidden = nn.Linear(input_size, hidden_size * 4)
        self.hidden2hidden = nn.Linear(hidden_size, hidden_size * 4)
        
    def forward(self, xt, a_prev, c_prev):
        '''
        Arguments:
        xt -- (batch, input_size)
        a_prev -- (batch, hidden_size) 
        c_prev -- (batch, hidden_size)
        '''
        gates = self.input2hidden(xt) + self.hidden2hidden(a_prev)
        update, forget, output, c_hat = gates.chunk(4, 1) # (batch, hidden_size)
        update = F.sigmoid(update)
        forget = F.sigmoid(forget)
        output = F.sigmoid(output)
        c_hat = F.tanh(c_hat)
        
        c = update.mul(c_hat) + forget.mul(c_prev)
        a = output.mul(c)
        
        return a, c

def test():        
    input_size = 5
    hidden_size = 10
    cell = LSTMCell(input_size, hidden_size)

    xt = torch.rand(3, input_size)
    a_prev = torch.rand(3, hidden_size)
    c_prev = torch.rand(3, hidden_size)

    at, ct = cell(xt, a_prev, c_prev)
    print(at, ct, sep='\n')

test()

tensor([[ 0.0085,  0.1264,  0.4562, -0.1187,  0.3414,  0.0417,  0.1139,  0.2361,
          0.2032, -0.0179],
        [-0.0082,  0.4311,  0.5325, -0.1125,  0.2105,  0.1196,  0.1316, -0.1792,
          0.1112,  0.0969],
        [ 0.0354,  0.2737,  0.6363,  0.0120,  0.1885,  0.0529,  0.1305, -0.0971,
          0.0577,  0.1370]], grad_fn=<MulBackward0>)
tensor([[ 0.0165,  0.2783,  0.5819, -0.3928,  0.9136,  0.2373,  0.3157,  0.5183,
          0.3081, -0.0601],
        [-0.0148,  0.7638,  0.6863, -0.3094,  0.5877,  0.7003,  0.2881, -0.3718,
          0.2005,  0.4167],
        [ 0.0694,  0.6167,  0.8097,  0.0307,  0.6021,  0.4673,  0.2800, -0.1842,
          0.1151,  0.5373]], grad_fn=<AddBackward0>)


LSTM和GRU的计算过程的对比, GRU相比LSTM的区别有:

- 用一个更新门代替更新门和遗忘门对细胞状态进行更新
- 将隐藏层和输出项合并为同一个向量
- 用于构造候选隐藏层的矩阵增加了r门

<img src="src/pictures/c5_w1_6.png">

## BiDirectional RNN

双向RNN的优点是能够获得前后的信息作为某一个时间步上的输出, 增加了信息输入, 能够提高预测的准确程度, 缺点是参数数量增加了一倍; 必须获得完整的序列才能开始学习.

<img src="src/pictures/c5_w1_7.png">

## Deep RNN

将下层的输出作为上层的输入, 对于某一个时间步上某一层的神经元, 它获得的信息来自同层的前一个时间步和同时间步的下方神经元.

<img src="src/pictures/c5_w1_8.png">